# Preprocessing

## Preparation

In [1]:
import re
import pyproj
import logging
import shapely
import rasterio
import threading
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from collections import namedtuple
from src.utils import get_data_dir
from collections import defaultdict
from src.decorators import benchmark
from IPython.display import clear_output
from rasterio import warp, merge, mask, windows, coords


# Convenient access to data directory, is a namedtuple with folder names as attributes
DIRS = get_data_dir(str(Path('data').resolve()))

WGS84 = {'init': 'epsg:4326'}

THREADLIMIT = 4

# init Logging
formater = logging.Formatter('%(asctime)s %(levelname)s: %(message)s', '%d/%m/%y %H:%M:%S')
handler = logging.FileHandler(str(DIRS.log / 'preprocessing.log'), 'a+')
handler.setLevel(logging.DEBUG)
handler.setFormatter(formater)

LOGGER = logging.getLogger('Preprocessing')
LOGGER.setLevel(logging.DEBUG)
LOGGER.addHandler(handler)


def read_raster(item):
    """
    Helper method to return a raster file as a opened instance of
    rasterio.io.DatasetReader in read mode.
    
    :param item: str, pathlib.Path or rasterio.io.DatasetReader
        Should be the path to the raster file on filesystem as a string
        or pathlib.Path object. If item is a instance of DatasetReader
        the function returns immediately.
    :return: rasterio.io.DatasetReader
        Retruns an instance of rasterio.io.DatasetReader in read mode.
    """
    if isinstance(item, rasterio.io.DatasetReader):
        return item
    else:
        try:
            path = str(item)  # Cast pathlib.Path to string
            return rasterio.open(path, 'r')
        except:
            msg = 'Attr {}, Type {} is not a valid raster file'.format(item, type(item))
            raise ValueError(msg)

# TODO docstring
def fetch_metadata(from_path_or_reader, *args):
    """
    This method fetches user selected metadata features from a raster file and
    returns them as a named tuple where the attribute name is the selected
    metadata feature key and the assigned value the corresponding metadata
    feature. Please refer to the documentation of rasterio for a comprehenisve 
    list of fetchable metadata features provided by a raster file.  
    
    :param from_path_or_reader: str, pathlib.Path or rasterio.io.DatasetReader
        Path to the raster file on drive as string or pathlib.Path object or a
        opened raster dataset.
    :param *args: str
        
    :return: namedtuple
        The requested metadata features as a namedtuple where the attribute 
        name is the selected metadata feature key and the assigned value the 
        corresponding metadata feature.
        Example:
        fetch_metadata(('bounds', 'crs'), path)
        (bounds=value, crs=value)
    """
    reader = read_raster(from_path_or_reader)
    
    values = []
    for f in args:
        value = reader.__getattribute__(f)
        if value is not None:
            values.append(value)
        else:
            raise ValueError('{} is not set'.format(f))
    
    # Closes the reader but if a user just want to proceed with the reader
    # provided as arg this can be a pitfal
    reader.close()
    
    if len(values) > 1:
        Metadata = namedtuple('Metadata', args)
        return Metadata(*values)
    return values[0]


def polygon_from(bounds):
    """
    Creates a polygon object from a bounds object. 
    
    :param bounds: namedtuple
        Should be a namedtuple containing the attributes
        left, right, top and bottom
    :return: shapely.geometry.Polygon
        The polygon object in extent of the provided bounds
        object.
    """
    x_points = ['left', 'left', 'right', 'right']
    y_points = ['top', 'bottom', 'bottom', 'top']    
    
    polygon_bounds = [
        (bounds.__getattribute__(x), bounds.__getattribute__(y))
        for x, y in zip(x_points, y_points)
    ]
    
    return shapely.geometry.Polygon(polygon_bounds)


def reproject_bounds(bounds, source_crs, target_crs):
    """
    This method reprojects the coordinates of an bounds object to the requested
    coordinate system.
    
    :param bounds: namedtuple
        Should be a namedtuple containing the attributes
        left, right, top and bottom.
    :param source_crs: dict
        The coordinate reference system of the bounds object as a dictionary
        with the following shape:
        {'init': 'epsg:<id>'} where <id> is the epsg number of the crs 
    :param target_crs: dict
        The coordinates system for the reprojection of the bounds object.
        Shape should be equal to source_crs.
    :return: namedtuple(left, right, top, bottom)
        Reprojected bounds object
    """
    p1 = pyproj.Proj(**source_crs)
    p2 = pyproj.Proj(**target_crs)
    
    left, bottom = pyproj.transform(p1, p2, bounds.left, bounds.bottom)
    right, top = pyproj.transform(p1, p2, bounds.right, bounds.top)
    
    BoundingBox = namedtuple('BoundingBox', 'left bottom right top')
    return BoundingBox(left, bottom, right, top)


# TODO docstring
def polygoniz(rasters, target_crs):
    """
    This function creates a tile index from a list of raster files.
    
    :param rasters: list
        Pending
    :param target_crs: dict
        If the raster files have different coordinate reference systems
        this arguement prevents a messed up dataset.
    :return: geopandas.GeoSeries
        Each element of the geoseries is a polygon
        covering the corresponding raster file.
    """
    polygons = []
    for raster in rasters:
        bounds, crs = fetch_metadata(raster, 'bounds', 'crs')
        if crs != target_crs:
            bounds = reproject_bounds(bounds, crs, target_crs)
        polygon = polygon_from(bounds)
        polygons.append(polygon)
        
    geometry = gpd.GeoSeries(polygons)
    geometry.crs = target_crs
    return geometry


# TODO docstring
def tile_index(rasters, target_crs, **kwargs):
    """
    Description Pending
    
    :param rasters: list
        A list of str where each element is a path to a raster file
        on disk.
    :param target_crs: dict
        The coordinate reference system which should be applied on
        the tile index dataset.
    :param **kwargs:
    :return: geopandas.GeoDataFrame
    """
    geometry = polygoniz(rasters, target_crs)
    features = pd.DataFrame(kwargs)
    
    return gpd.GeoDataFrame(features, geometry=geometry)


# TODO accept **kwargs to alter write parameters docstring
def reproject_from(in_path, to_crs, out_path):
    """
    This method reprojects a raster file to a selected coordinate
    reference system.
    
    :param in_path: str
        Path to raster file on drive
    :param to_crs: dict
        Target coordinate reference system for reprojection
    :param out_path: str
        Path where the reprojected raster file should be stored
    :return: str
        Path where the reprojected raster file is stored
    """
    with rasterio.open(in_path, 'r') as src:
        affine, width, height = rasterio.warp.calculate_default_transform(
            src_crs=src.crs,
            dst_crs=to_crs,
            width=src.width,
            height=src.height,
            **src.bounds._asdict(),
        )
        
        kwargs = src.profile.copy()
        kwargs.update(
            transform=affine,
            width=width,
            height=height,
            crs=to_crs
        )
        
        with rasterio.open(out_path, 'w', **kwargs) as dst:
            for idx in src.indexes:
                rasterio.warp.reproject(
                    source=rasterio.band(src, idx), 
                    destination=rasterio.band(dst, idx)
                )
        
        return out_path


# TODO docstring
def reproject_like(template, in_path, out_path: str):
    """
    """
    crs, transform, width, height = fetch_metadata(template, 'crs', 'transform', 'width', 'height')
    
    with rasterio.open(in_path, 'r') as src:
        out_kwargs = src.profile.copy()
        out_kwargs.update({
            'crs': crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        with rasterio.open(out_path, 'w', **out_kwargs) as dst:
            rasterio.warp.reproject(source=rasterio.band(src, list(range(1, src.count + 1))), 
                                    destination=rasterio.band(dst, list(range(1, src.count + 1))))
    
    return out_path


def merge_from(rasters, **kwargs):
    """
    Merges a list of raster files to one single raster dataset.
    This method is wrapped around the rasterio.merge.merge method
    therefore this method accept keyword arguments as well.
    
    :param rasters: list
        A list of strings or pathlib.Path objects where each list element reference a path to a
        raster file on drive.
    :param **kwargs:
        Please refer to the rasterio documentation for a full list
        of possible keyword arguments.
    :return: namedtuple(data, affine)
        A namedtuple with the attributes data and affine, where the parameter
        data contains the merged data of the raster files as a numpy.ndarray 
        and affine an affine transformation matrix.
    """
    readers = [read_raster(raster) for raster in rasters]

    dest, affine = rasterio.merge.merge(readers, **kwargs)
    
    [reader.close() for reader in readers]
    Merge = namedtuple('Merge', 'data affine')  
    return Merge(dest, affine)


def merge_alike(with_template, to_merge):
    """
    Merges the input raster files like a template raster, hence the output
    dataset has same bounds and resolution as the template raster. Both datasets
    must have the same coordinate reference system.
    
    :param with_template: str
        Path to the template raster file
    :param to_merge: list
        A list of strings where each list element reference a path to a
        raster file on drive.
    :return: namedtuple(data, affine)
        A namedtuple with the attributes data and affine, where the parameter
        data contains the merged data of the raster files as a numpy.ndarray 
        and affine an affine transformation matrix.
    """
    bounds, res = fetch_metadata(with_template, 'bounds', 'res')
    return merge_from(to_merge, bounds=bounds, res=res)


def write(data, to_path, **kwargs):
    """
    Writes a multi-dimensional numpy.ndarray as a raster dataset to file.
    This method is wrapped around the rasterio.open method therefore 
    you can modify the methods behavior  with **kwargs arguements provided
    by the rasterio documentation.
    
    :param data: numpy.ndarray
        A multi-dimensional numpy array. If array has three dimensions
        each dimension depict a raster band. If array has two dimensions
        the resulting raster file contains a sinlge band.
    :param to_path: str
        Path where the new raster file should be stored
    :param **kwargs:
        Keyword arguments consumed by the rasterio.open function.
        Please refer to the rasterio documentation for a comprehensive
        list of possible keyword arguements.
    :return: str
        Path where the raster file is stored 
    """
    if len(data.shape) == 3:
        idx, height, width = data.shape  # z, y, x
    elif len(data.shape) == 2:
        idx = 1  # z
        height, width = data.shape  # y, x
        data = np.reshape(data.copy(), (idx, height, width))
    else:
        raise ValueError('Please, provide a valid dataset')
    
    dtype = data.dtype
    kwargs.update(
        count=idx,
        height=height,
        width=width,
        dtype=dtype
    )
    
    with rasterio.open(to_path, 'w', **kwargs) as dst:
        for i in range(idx):
            dst.write(data[i], i+1)  # rasterio band index start at one, thus we increment by one
    
    return to_path


def int_to_orient(x, y):
    """
    Converts a x- and y-coordinate to an integer north/south,
    weste/east string representation.
    Example: (x=-179.3457, y=80.2222) -> 80N_179W
             
    :param x: float
        Longitudinal coordinate  
    :param y: float
        Latitudinal coordinate 
    :return: str
        Lat/Lon coordinates as a integer string with the according
        orientation.
    """
    x = round(x)
    y = round(y)
    
    lng, we = (-1 * x, 'W') if x < 0 else (x, 'E')
    lat, ns = (-1 * y, 'S') if y < 0 else (y, 'N')
    
    return '{:02d}{}_{:03d}{}'.format(lat, ns, lng, we)


# TODO docstring
def round_bounds(bounds):
    """
    """
    attrs = ('left', 'bottom', 'right', 'top')
    
    coords = []
    for attr in attrs:
        coord = bounds.__getattribute__(attr)
        coords.append(round(coord))
    
    BoundingBox = namedtuple('BoundingBox', attrs)
    return BoundingBox(*coords)


# TODO docstring
def clip_raster(raster, dst_bounds):
    """
    """
    src = read_raster(raster)
    src_bounds = src.bounds
    
    if coords.disjoint_bounds(src_bounds, dst_bounds):
        msg = 'Raster bounds {} are not covered by clipping bounds {}'.format(src_bounds, dst_bounds)
        raise ValueError(msg)
    
    window = src.window(*dst_bounds)
    window = window.round_lengths(op='ceil')
    transform = src.window_transform(window)
    data = src.read(window=window, out_shape=(src.count, window.height, window.width))
    
    src.close()
    return data, transform


# TODO refactor rename to dispatch_filename docstring
def dispatch_name(val, key, idx):
    """
    """
    return {
        'merge_0': lambda: ('cover', '{}_{}.tif'.format(idx, key)),
        'merge_1': lambda: ('loss', '{}_{}.tif'.format(idx, key)),
        'merge_2': lambda: ('gain', '{}_{}.tif'.format(idx, key)),
        'merge_3': lambda: ('biomass', '{}_{}.tif'.format(idx, key)),
        'merge_4': lambda: ('confidence', '{}_{}.tif'.format(idx, key)),
        'reproject_0': lambda: ('gl30_10', '{}_{}.tif'.format(idx, key)),
        'reproject_1': lambda: ('gl30_00', '{}_{}.tif'.format(idx, key)),
        'reproject_2': lambda: ('soil', '{}_{}.tif'.format(idx, key)),
    }.get(val, None)()


# TODO docstring
def dispatch_range(key, value):
    """
    """
    return {
        'cover': lambda: not 0 <= value <= 100, # union should be not greater than 101
        'loss': lambda: not 0 <= value <= 12, # union should be not greater than 13
        'gain': lambda: not 0 <= value <= 1,  # union should be not greater than two
        'biomass': lambda: not -32768 <= value <= 1000,
        'confidence': lambda: None,
        'soil': lambda: None,
        'gl30_00': lambda: not 0 <= value <= 255,
        'gl30_10': lambda: not 0 <= value <= 255 ,
    }.get(key, lambda: False)()


# TODO docstring
def test_bounds_are_equal(datasets):
    """
    """
    bounds = fetch_metadata(datasets[0][1], 'bounds') 
    data_bounds = [fetch_metadata(item[1], 'bounds') for item in datasets]

    for idx, db in enumerate(data_bounds):
        if bounds != db:
            msg = '"{}" inequal bounds "{} != {}"'.format(datasets[idx].name, db, bounds)
            LOGGER.warning(msg)


# TODO docstring
# broken
# set operation intersection length = 0 non overlap
def test_expected_valuerange(key, dataset):
    """
    """
    raster = read_raster(dataset)
    values = np.unique(raster.read())
    raster.close()
    
    for val in values:
        if dispatch_range(key, val):
            msg = '"{}" with range "{}" is not expected range for "{}"'.format(dataset.name, values, key)
            LOGGER.warning(msg)


def test_expected_metadata():
    pass


def test_landmass_overlapping():
    pass


# TODO refactor and docstring
def worker(to_reproject: list, to_crs: dict, to_merge_alike: list, out_path: str, generic_name: str):
    """
    """
    template = None
    path = Path(out_path)
    
    for idx, raster in enumerate(to_reproject):
        opath = str(path / 'reproject_{}_{}'.format(idx, generic_name))
        
        if idx == 0:
            try:
                template = reproject_from(raster, to_crs, opath)
            except Exception as err:
                LOGGER.error('Fatal no template %s', raster, exc_info=err)
                raise err
        else:
            try:
                reproject_like(template, raster, opath)
            except Exception as err:
                LOGGER.warning('Unable to reproject %s', raster, exc_info=err)
                    
    kwargs = fetch_metadata(template, 'profile')
    
    for idx, rasters in enumerate(to_merge_alike):
        opath = str(path / 'merge_{}_{}'.format(idx, generic_name))       
        try:
            data, transform = merge_alike(template, rasters)
            kwargs.update({'transform': transform})
            write(data, opath, **kwargs)
        except Exception as err:
            LOGGER.warning('Unable to merge %s, cant create %s', rasters, opath)
            

# TODO refactor and docstring        
def clip_worker(to_clip: list, bounds: namedtuple, profile: dict, out_path: str):
    """
    """
    key = int_to_orient(bounds.left, bounds.top)
    path = Path(out_path)
    
    for idx, raster in enumerate(to_clip):
        data, transform = clip_raster(raster, bounds)
        opath = path / '{}_{}.tif'.format(idx, key)
        profile.update({'transform': transform})
        write(data, str(opath), **profile)

## Masking

### GFC mask

In [2]:
gfc = sorted(DIRS.gfc.glob('*.tif'))

data_len = int(len(gfc)/3)

kwargs = {
    'gain': [i.name for i in gfc[:data_len]],
    'loss': [i.name for i in gfc[data_len:2*data_len]],
    'cover': [i.name for i in gfc[2*data_len:]],
}

gfc_mask = tile_index(gfc[:data_len], WGS84, **kwargs)
gfc_mask.to_file(str(DIRS.masks / 'gfc_mask.shp'))
gfc_mask.head()

,cover,gain,loss,geometry
0,Hansen_GFC2013_treecover2000_00N_000E.tif,Hansen_GFC2013_gain_00N_000E.tif,Hansen_GFC2013_lossyear_00N_000E.tif,POLYGON ((-0.0001388888888982365 0.00013888888...
1,Hansen_GFC2013_treecover2000_00N_010E.tif,Hansen_GFC2013_gain_00N_010E.tif,Hansen_GFC2013_lossyear_00N_010E.tif,POLYGON ((9.999861111111102 0.0001388888888840...
2,Hansen_GFC2013_treecover2000_00N_010W.tif,Hansen_GFC2013_gain_00N_010W.tif,Hansen_GFC2013_lossyear_00N_010W.tif,POLYGON ((-10.0001388888889 0.0001388888888840...
3,Hansen_GFC2013_treecover2000_00N_020E.tif,Hansen_GFC2013_gain_00N_020E.tif,Hansen_GFC2013_lossyear_00N_020E.tif,POLYGON ((19.9998611111111 0.00013888888888402...
4,Hansen_GFC2013_treecover2000_00N_020W.tif,Hansen_GFC2013_gain_00N_020W.tif,Hansen_GFC2013_lossyear_00N_020W.tif,POLYGON ((-20.0001388888889 0.0001388888888840...


### GL30 mask

In [3]:
gl30 = sorted(DIRS.gl30.glob('*.tif'), key=lambda key: (key.name[7:11], key.name[0:6]))

exclude = 'n01_00 s01_00 s01_10 s01_15 s01_20 s60_00 s60_05 s60_10 s60_15 n53_00'.split()
gl30 = [item for item in gl30 if item.name[0:6] not in exclude]
data_len = int(len(gl30)/2)

kwargs = {
    'gl30_00': [i.name for i in gl30[:data_len]],
    'gl30_10': [i.name for i in gl30[data_len:]],
    'key': [i.name[0:6] for i in gl30[:data_len]]
}

gl30_mask = tile_index(gl30[data_len:], WGS84, **kwargs)
gl30_mask.to_file(str(DIRS.masks / 'gl30_mask.shp'))
gl30_mask.head()

,gl30_00,gl30_10,key,geometry
0,n02_15_2000lc030.tif,n02_15_2010lc030.tif,n02_15,POLYGON ((-174.0053601744084 20.00401663536249...
1,n03_05_2000lc030.tif,n03_05_2010lc030.tif,n03_05,POLYGON ((-168.0054833302891 10.00519024901941...
2,n03_20_2000lc030.tif,n03_20_2010lc030.tif,n03_20,POLYGON ((-168.0051433812486 25.00312959291788...
3,n04_00_2000lc030.tif,n04_00_2010lc030.tif,n04_00,POLYGON ((-162.0055192236557 5.005478418984219...
4,n04_05_2000lc030.tif,n04_05_2010lc030.tif,n04_05,"POLYGON ((-162.0054833302891 10.0051902490194,..."


### Biomass mask

In [4]:
biomass = gpd.read_file(str(DIRS.masks / 'biomass.geojson'))
biomass_mask = biomass.drop(biomass.columns[[0, 1, 4, 5]], axis=1)
biomass_mask.rename(columns={'download': 'biomass'}, inplace=True)

for idx, row in biomass_mask.iterrows():
    biomass = row.biomass.split('/')[-1]
    confidence = row.confidence.split('/')[-1]
    
    biomass_mask.at[idx, 'biomass'] = biomass
    biomass_mask.at[idx, 'confidence'] = confidence

biomass_mask.to_file(str(DIRS.masks / 'biomass_mask.shp'))
biomass_mask.head()

,biomass,confidence,geometry
0,10N_090W_merge.tif,merged_per_tropical_asia_10N_090W.tif,POLYGON ((-90.0001388885744 0.0001388894243932...
1,10N_050E_merge.tif,merged_per_tropical_asia_10N_050E.tif,"POLYGON ((49.9998611109019 7.999861110840482, ..."
2,10S_170E_merge.tif,merged_per_tropical_asia_10S_170E.tif,POLYGON ((169.9998611109663 -12.00013888921919...
3,20N_100W_merge.tif,merged_per_tropical_asia_20N_100W.tif,POLYGON ((-100.0001388891786 13.00013888876461...
4,20S_020E_merge.tif,merged_per_tropical_asia_20S_020E.tif,POLYGON ((19.99986111088579 -29.99986111075941...


## Raster alignment
- include soil layer

In [ ]:
gl30_mask = gpd.read_file(str(DIRS.masks / 'gl30_mask.shp'))
gfc_mask = gpd.read_file(str(DIRS.masks / 'gfc_mask.shp'))
biomass_mask = gpd.read_file(str(DIRS.masks / 'biomass_mask.shp'))

intersect = gpd.overlay(gfc_mask, gl30_mask, how='intersection')
intersect = gpd.overlay(intersect, biomass_mask, how='intersection')

threads = []
for key, values in intersect.groupby(by='key', sort=False):
    if len(threads) == THREADLIMIT:
        [thread.join() for thread in threads]
        threads = []
    
    to_reproject = [
        str(DIRS.gl30 / name)
        for name in list(*zip(set(values.gl30_10), set(values.gl30_00)))
    ]
    to_reproject.append(str(DIRS.soil / 'GSOCmapV1.1.tif'))
    to_merge = [
        [str(DIRS.gfc / name) for name in set(values.cover)],
        [str(DIRS.gfc / name) for name in set(values.loss)],
        [str(DIRS.gfc / name) for name in set(values.gain)],
        [str(DIRS.biomass / name) for name in set(values.biomass)],
        [str(DIRS.biomass / name) for name in set(values.confidence)],
    ]
    generic_name = '{}.tif'.format(key)
    
    thread = threading.Thread(target=worker,
                              args=(to_reproject, WGS84, to_merge, str(DIRS.proc), generic_name))
    thread.start()
    threads.append(thread)

## Croping and masking

In [2]:
files = defaultdict(list)
regex = re.compile(r'.*(?P<key>(?:n|s)\d{2}_\d{2}).*', re.I)

for path in DIRS.proc.glob('*.tif'):
    match = regex.match(str(path))
    files[match.group('key')].append(path)
    files[match.group('key')] = sorted(files[match.group('key')])

threads = []
features = []
polygons = []
length = len(files)
idx = 0
for key, values in files.items():
    if len(threads) == THREADLIMIT:
        clear_output()
        
        [thread.join() for thread in threads]
        threads = []
        
        ratio = round((idx / length) * 100, 2)
        print('Processed {} % of {} %'.format(ratio, 100))
    
    bounds, profile = fetch_metadata(values[0], 'bounds', 'profile')
    bounds = round_bounds(bounds)

    thread = threading.Thread(target=clip_worker, args=(values, bounds, profile, str(DIRS.proc),))
    thread.start()
    threads.append(thread)
    
    key = int_to_orient(bounds.left, bounds.top)
    feature = dict([dispatch_name('{0[0]}_{0[1]}'.format(item.name.split('_')), key, idx)
                    for idx, item in enumerate(values)])
    feature['key'] = key
    features.append(feature)
    polygons.append(polygon_from(bounds))
    
    idx += 1

geometry = gpd.GeoSeries(polygons)
df = pd.DataFrame(features)
layer = gpd.GeoDataFrame(df, geometry=geometry)
layer.crs = WGS84

layer.to_file(str(DIRS.masks / 'final_mask.shp'))

## Testing

In [57]:
# BUGS
final_mask = gpd.read_file(str(DIRS.masks / 'final_mask.shp'))
columns = final_mask.columns[:-1]

for idx, vals in final_mask.iterrows():
    gen = filter(lambda x: x[1].split('.')[-1] == 'tif', zip(columns, vals[:-1]))
    data = [(key, DIRS.proc / path) for key, path in gen]

    test_bounds_are_equal(data)
    [test_expected_valuerange('cover', path) for key, path in data]


cover /media/ilex/StorageOne/docs/code/python/projects/Master/data/proc/3_20S_048W.tif
cover /media/ilex/StorageOne/docs/code/python/projects/Master/data/proc/0_20S_048W.tif
cover /media/ilex/StorageOne/docs/code/python/projects/Master/data/proc/2_20S_048W.tif
cover /media/ilex/StorageOne/docs/code/python/projects/Master/data/proc/5_20S_048W.tif
cover /media/ilex/StorageOne/docs/code/python/projects/Master/data/proc/4_20S_048W.tif
cover /media/ilex/StorageOne/docs/code/python/projects/Master/data/proc/1_20S_048W.tif
